In [10]:
import os
from osgeo import gdal,ogr, osr
gdal.UseExceptions()

In [2]:
source = gdal.Open("/vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/SCL.tif")

gt_forward=source.GetGeoTransform()

scl = source.GetRasterBand(1)

In [12]:
def pixel_to_coords(geo_transform, x, y):
    
    x_min = geo_transform[0]
    x_size = geo_transform[1]
    y_min = geo_transform[3]
    y_size = geo_transform[5]
    px = x * x_size + x_min 
    py = y * y_size + y_min 
    
 
    srs = osr.SpatialReference()
    srs.ImportFromWkt(source.GetProjection())
    
    srs_4326 = srs.CloneGeogCS()
    ct = osr.CoordinateTransformation(srs, srs_4326)
    
    long, lat, height = ct.TransformPoint(px, py)
    
    return long, lat

In [14]:
pixel_to_coords(gt_forward, 100, 100)

Warning 1: PROJ: proj_create_from_database: Open of /opt/conda/envs/env_label/share/proj failed
Warning 1: The definition of projected CRS EPSG:32610 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.


(-121.79397555246852, 40.626541370770724)

In [15]:

field_name = 'class'
field_type = ogr.OFTInteger

# Create the output Driver
out_driver = ogr.GetDriverByName('GeoJSON')

# Create the output GeoJSON
out_datasource = out_driver.CreateDataSource('labels.geojson')
out_layer = out_datasource.CreateLayer('labels.geojson', geom_type=ogr.wkbPolygon )
id_field = ogr.FieldDefn(field_name, field_type)
out_layer.CreateField(id_field)
# Get the output Layer's Feature Definition
feature_def = out_layer.GetLayerDefn()

for px in range(0, 5490, 240):
    
    for py in range(0, 5490, 240):
    
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(*pixel_to_coords(gt_forward, px, py))
        
        # create a new feature
        out_feature = ogr.Feature(feature_def)

        # Set new geometry
        out_feature.SetGeometry(point)
        out_feature.SetField(field_name, int(scl.ReadAsArray(px, py, 1, 1)[0][0]))
        # Add new feature to output Layer
        out_layer.CreateFeature(out_feature)

        # dereference the feature
        out_feature = None

# Save and close DataSources
out_datasource = None